# Intro
In this notebook, we briefly demonstrate how you can use pre-built models from the [open model hub](https://www.objectiv.io/docs/open-model-hub/) in conjunction with our [modeling library](https://objectiv.io/docs/modeling/) to quickly build model stacks to answer common product analytics questions.

This example uses real, unaltered data that was collected from objectiv.io with Objectiv’s Tracker. All models in the open model hub are compatible with datasets that have been validated against the [open analytics taxonomy](https://objectiv.io/docs/taxonomy/).

For an overview of all available models, check out the [open model hub docs](https://www.objectiv.io/docs/open-model-hub/).

In [ ]:
from bach_open_taxonomy import ObjectivFrame

# Instantiate the data object
As a first step, the Objectiv Frame object is instantiated. The Objectiv Frame is an extension to Bach DataFrame to use specifically for data that was collected with Objectiv’s Tracker. Bach is Objectiv’s data modeling library. With Bach, you can use familiar Pandas-like DataFrame operations in your notebook. It uses a SQL abstraction layer that enables models to run on the full dataset, and you can output models to SQL with a single command.

The Objectiv Bach api is heavily inspired by the pandas api. We believe this provides a great, generic interface to handle large amounts of data in a python environment while supporting multiple data stores. For more details on Objectiv Bach, visit the docs.

It loads the data as stored by the Objectiv Tracker, makes a few transformations, and sets the right data types.

This object points to the data on which the models from the open model hub will be applied. The `time_aggregation` parameter determines the standard timeframe that is used with aggregation functions from the model hub. Ie. 'YYYY-MM-DD' means that days are used for the time aggregation. Only data starting at `start_date` is used for all following operations.

**Note**  
All operation and models in this notebook are run directly on entire data set in the SQL database using Bach. While the api resembles pandas, pandas is _not_ used for any the operations and calculations.

In [ ]:
df = ObjectivFrame.from_objectiv_data(time_aggregation='YYYY-MM-DD', start_date='2022-01-04')

# Using the open model hub
The ModelHub contains a growing collection of open-source, free to use data models that you can take, chain and run to quickly build highly specific model stacks for product analysis and exploration. It includes models for a wide range of typical product analytics use cases. The source is available for all models and you're free to make any changes to them. 

All models are in the `.model_hub` namespace of the Objectiv Frame. You can also use `.mh` as a shorthand. 

The model hub has two main type of functions: `map` and `aggregate`. 
* `map` functions always return a series with the same shape and index as the Objectiv Frame they originate from. This ensures they can be added as a column to that Objectiv Frame.
* `aggregate` fuctions return aggregated data in some form from the Objectiv Frame. Can also be accessed with `agg`.

Additionally the Objectiv Frame can be filtered with the `filter` method. `map` functions that return SeriesBoolean can be used with the `filter` method.

This notebook demonstrates how to use the model hub by showcasing a selection of the models from the model hub.

## A simple aggregation model
Below we show two ways of calculating the unique users per day. First we calculate it using the pandas-inspired Bach api. With `.head()` we immediately query the data to show the results. `.to_pandas()` can be used to use all results as a pandas object in python.

In [ ]:
df['date'] = df.moment.dt.sql_format('YYYY-MM-DD')
df.groupby('date').user_id.nunique().head(10)

Now, we calculate the same by using an aggregation model from the model hub. It uses the *time_aggregation* that is set when the Objectiv Frame was instantiated.

In [ ]:
users = df.model_hub.aggregate.unique_users()
users.head(10)

The model hub allows us to run models with a single line of code on the entire data set. This makes it possible to simplify, reuse and chain data models as the example above, and the following examples show. 

Bach and the model hub work seamlessly together, so the Bach api can always be used with the Objectiv Frame and model hub outputs.

## Export to metabase
All models' results from the model hub can be exported to Metabase to visualize or share the model results.

In [ ]:
# df.model_hub.to_metabase(users, config={'name': 'Unique Users'})

## Using `map` with the model hub & combining models
This example shows how you use map to label users as a new user. This uses *time_aggregation*. As *time_aggregation* was set to 'YYYY-MM-DD' it means all hits are labeled as new for the entire day in which the user had its first session.

In [ ]:
is_new_user = df.model_hub.map.is_new_user()
is_new_user.head(10)

Or we can label conversion events. To do this we first have to define what a conversion is by setting the type of event and the location on the product at which this event was triggered with `add_conversion_event` (this is called the location stack, see [here](open-taxonomy-how-to.ipynb#global_contexts-&-location_stack) for info).

In [ ]:
df.add_conversion_event(location_stack=df.location_stack.json[{'id': 'Quickstart Guide', '_type': 'LinkContext'}:],
                        event_type='PressEvent',
                        name='quickstart_presses')
conversion_events = df.mh.map.is_conversion_event('quickstart_presses')
conversion_events.head(10)

### Map, filter, aggregate
As the map functions above retured a SeriesBoolean, they can be used in the model hub combined with filter and aggregation models. In the example below, the average session duration is calculated for new users.

In [ ]:
duration_new_users = df.model_hub.filter(is_new_user).model_hub.aggregate.session_duration()
duration_new_users.head(10)

Or, as another example, converted unique users

In [ ]:
df.model_hub.filter(conversion_events).model_hub.aggregate.unique_users().head(10)

### Combining model results
Results from aggregation models can be used together if they share the same index type (similar to pandas). In this example the share of new users per day is calculated.

In [ ]:
new_user_share = df.mh.filter(is_new_user).mh.agg.unique_users() / df.mh.agg.unique_users()
new_user_share.head(10)

### Chaining model filters

Filter model results can be chained, as the output is always an ObjectivFrame with the same columns but less (or equal the) number of rows as the initial ObjectivFrame. In this example we calculate the number of users that were new in a month and also that converted twice on a day. 

In [ ]:
# stackin': unique new users that converted three times in a session
df.mh.filter(df.mh.map.is_new_user(time_aggregation = 'YYYY-MM'))\
  .mh.filter(df.mh.map.conversion_count(name='quickstart_presses')==2)\
  .mh.agg.unique_users()\
  .head()

This concludes the open model hub demo.

We hope you’ve gotten a taste of the power and flexibility of the open model hub to quickly answer common product analytics questions. You can take it a lot further and build highly specific model stacks for in-depth analysis and exploration.

For a complete overview of all available and upcoming models, check out the [open model hub docs](https://objectiv.io/docs/open-model-hub).

The Bach modeling library can also do a whole lot more. It includes a specific set of operations that are designed for effective modeling on datasets that embrace the open analytics taxonomy, and pandas-compatability means you can tap into the rich ML ecosystem that pandas is well-known for.

There is another notebook in the same folder that demonstrates what you can do with the Bach modeling library [open-taxonomy-how-to.ipynb](open-taxonomy-how-to.ipynb)